# Use Snowpark_OpenAI Notebook template

In [1]:
!!pip install "snowflake-connector-python[pandas]"

['Collecting snowflake-connector-python[pandas]',
 '  Using cached https://files.pythonhosted.org/packages/f8/c7/03d2ca5e460acb3a31d70ee17584fbc6fe828a52c9f83bbb20a392b8988b/snowflake_connector_python-3.7.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl',
 'Collecting requests<3.0.0',
 '  Using cached https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl',
 'Collecting asn1crypto<2.0.0,>0.24.0',
 '  Using cached https://files.pythonhosted.org/packages/c9/7f/09065fd9e27da0eda08b4d6897f1c13535066174cc023af248fc2a8d5e5a/asn1crypto-1.5.1-py2.py3-none-any.whl',
 'Collecting idna<4,>=2.5',
 '  Using cached https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl',
 'Collecting certifi>=2017.4.17',
 '  Using cached https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c96257acb4fce61e7de14/cer

In [2]:
!pip install -q matplotlib seaborn xgboost

ERROR: snowflake-ml-python 1.0.10 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.25.3 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting

import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import datetime

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

# Read Customer and Payment data from Snowflake respective tables

In [2]:
#Import all snowflake connection details from template variables.

db_user = os.getenv('sf_user')
db_password =  'Password@2023' #os.getenv('sf_password')
db_account = os.getenv('sf_account')
db_database =  os.getenv('sf_db')
db_role = os.getenv('sf_role')
db_warehouse = 'FOSFOR_FDC'
db_schema = 'PUBLIC'

In [3]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [4]:
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

cc_customer = session.table('FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA').to_pandas()
cc_payment = session.table('FDC_DATA_MANISH.PUBLIC.CC_PAYMENT_DATA').to_pandas()

# Run Descriptive Statistics

In [5]:
print('Size of Custommer data', cc_customer.shape)
print('Size of Payment details', cc_payment.shape)

Size of Custommer data (1125, 13)
Size of Payment details (8250, 12)


In [6]:
cc_customer.head()

,LABEL,ID,FEA_1,FEA_2,FEA_3,FEA_4,FEA_5,FEA_6,FEA_7,FEA_8,FEA_9,FEA_10,FEA_11
0,1.0,54982665.0,5.0,1245.5,3.0,77000.0,2.0,15.0,5.0,109.0,5.0,151300.0,244.948974
1,0.0,54989781.0,4.0,1415.0,3.0,93000.0,2.0,8.0,5.0,113.0,4.0,151300.0,273.861279
2,0.0,58998054.0,4.0,1250.0,3.0,137000.0,2.0,8.0,5.0,90.0,5.0,72000.0,1.000000
3,0.0,58995381.0,7.0,1220.0,3.0,76000.0,2.0,11.0,2.0,90.0,5.0,71002.0,1.000000
4,0.0,58987926.0,5.0,0.0,2.0,50000.0,2.0,15.0,5.0,108.0,4.0,450000.0,169.000000


In [7]:
cc_payment.head()

,ID,OVD_T1,OVD_T2,OVD_T3,OVD_SUM,PAY_NORMAL,PROD_CODE,PROD_LIMIT,UPDATE_DATE,NEW_BALANCE,HIGHEST_BALANCE,REPORT_DATE
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,0.0,31/12/9999
1,58995151,0,0,0,0,1,5,0.0,04/12/2016,588720.0,491100.0,31/12/9999
2,58997200,0,0,0,0,2,5,0.0,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,0.0,03/12/2016,15147.6,0.0,26/04/2016


In [8]:
cc_customer.describe(include='all')

,LABEL,ID,FEA_1,FEA_2,FEA_3,FEA_4,FEA_5,FEA_6,FEA_7,FEA_8,FEA_9,FEA_10,FEA_11
count,1125.000000,1.125000e+03,1125.000000,1125.000000,1125.000000,1.125000e+03,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000
mean,0.200000,5.783677e+07,5.482667,1113.864444,2.333333,1.208836e+05,1.928889,10.872000,4.832889,100.802667,4.195556,164618.495111,134.999004
std,0.400178,1.817150e+06,1.383338,438.066265,0.878773,8.844523e+04,0.257125,2.676437,2.971182,11.988955,0.855679,152520.488281,112.616798
min,0.000000,5.498235e+07,1.000000,0.000000,1.000000,1.500000e+04,1.000000,3.000000,-1.000000,64.000000,1.000000,60000.000000,1.000000
25%,0.000000,5.499050e+07,4.000000,1229.000000,1.000000,7.200000e+04,2.000000,8.000000,5.000000,90.000000,3.000000,60044.000000,1.000000
50%,0.000000,5.898975e+07,5.000000,1271.000000,3.000000,1.020000e+05,2.000000,11.000000,5.000000,105.000000,4.000000,72000.000000,173.205081
75%,0.000000,5.899799e+07,7.000000,1310.000000,3.000000,1.390000e+05,2.000000,11.000000,5.000000,111.000000,5.000000,151307.000000,202.484567
max,1.000000,5.900624e+07,7.000000,1481.000000,3.000000,1.200000e+06,2.000000,16.000000,10.000000,115.000000,5.000000,650070.000000,707.106781


In [9]:
cc_payment.describe(include='all')

,ID,OVD_T1,OVD_T2,OVD_T3,OVD_SUM,PAY_NORMAL,PROD_CODE,PROD_LIMIT,UPDATE_DATE,NEW_BALANCE,HIGHEST_BALANCE,REPORT_DATE
count,8.250000e+03,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,8250,8.250000e+03,8.250000e+03,8250
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3042,NaN,NaN,1863
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31/12/9999,NaN,NaN,31/12/9999
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,NaN,NaN,1114
mean,5.782173e+07,0.249091,0.127152,0.369212,187.681697,14.526667,8.232000,22170.138800,NaN,1.054042e+05,2.083356e+05,NaN
std,1.822724e+06,1.250197,0.860046,2.900320,1804.232570,12.053627,3.533055,53277.739379,NaN,1.887704e+06,2.744287e+06,NaN
min,5.498235e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,-4.030320e+04,0.000000e+00,NaN
25%,5.499050e+07,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,0.000000,NaN,0.000000e+00,2.050000e+04,NaN
50%,5.898905e+07,0.000000,0.000000,0.000000,0.000000,11.000000,10.000000,0.000000,NaN,0.000000e+00,4.086100e+04,NaN
75%,5.899655e+07,0.000000,0.000000,0.000000,0.000000,25.000000,10.000000,11000.000000,NaN,2.494800e+04,1.005000e+05,NaN


# Reverse Data Engineering to get Raw data

In [10]:
# checking missing data
def get_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    percent = round(percent,2)
    missing_df  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print( missing_df.head(20))
    return None

        Total  Percent
LABEL       0      0.0
ID          0      0.0
FEA_1       0      0.0
FEA_2       0      0.0
FEA_3       0      0.0
FEA_4       0      0.0
FEA_5       0      0.0
FEA_6       0      0.0
FEA_7       0      0.0
FEA_8       0      0.0
FEA_9       0      0.0
FEA_10      0      0.0
FEA_11      0      0.0
